In [1]:
import numpy as np

# Character to number mapping
char_to_num = {char: i for i, char in enumerate("ABCDEFGHIJKLMNOPQRSTUVWXYZ ?!")}
num_to_char = {i: char for char, i in char_to_num.items()}
mod = len(char_to_num)  # Modular base (29 characters)

# Function to compute the inverse of a matrix (non-modular)
def compute_inverse(matrix):
    determinant = int(round(np.linalg.det(matrix)))  # Determinant of the matrix
    if determinant == 0:
        raise ValueError("The key matrix is non-invertible (determinant is 0).")

    adjugate_matrix = np.round(np.linalg.inv(matrix) * determinant).astype(int)  # Adjugate matrix
    inverse_matrix = adjugate_matrix / determinant  # Divide by determinant
    return inverse_matrix

# Encoding function
def encode_message(message, key_matrix):
    message = message.upper()
    numbers = [char_to_num[char] for char in message]

    # Determine the size of the key matrix
    matrix_size = int(np.sqrt(len(key_matrix)))

    # Pad the message so its length is a multiple of the key matrix size
    while len(numbers) % matrix_size != 0:
        numbers.append(char_to_num[" "])  # Use space as padding

    # Group numbers based on the matrix size
    matrices = [numbers[i:i + matrix_size] for i in range(0, len(numbers), matrix_size)]

    # Multiply each group by the key matrix
    key_matrix = np.array(key_matrix).reshape(matrix_size, matrix_size)
    encoded_matrices = [(key_matrix @ np.array(group)) % mod for group in matrices]

    # Flatten the encoded result
    encoded_message = [num for mat in encoded_matrices for num in mat]
    return ' '.join(map(str, encoded_message))  # Return as space-separated numbers

# Decoding function
def decode_message(encoded_message, key_matrix, is_inverse_given):
    # Determine the matrix size
    matrix_size = int(np.sqrt(len(key_matrix)))

    # Reshape the key matrix
    key_matrix = np.array(key_matrix).reshape(matrix_size, matrix_size)

    # Compute the inverse if not given
    if not is_inverse_given:
        print("Computing inverse matrix...")
        key_matrix_inverse = compute_inverse(key_matrix)
        print("Computed inverse matrix (rounded to integers):")
        print(np.round(key_matrix_inverse))
    else:
        key_matrix_inverse = key_matrix

    # Group encoded message into matrix-size groups
    matrices = [encoded_message[i:i + matrix_size] for i in range(0, len(encoded_message), matrix_size)]

    # Decode by multiplying each group by the inverse key matrix
    decoded_matrices = [np.round(key_matrix_inverse @ np.array(group)).astype(int) % mod for group in matrices]

    # Flatten and map numbers back to characters
    decoded_numbers = [int(num) for mat in decoded_matrices for num in mat]
    decoded_message = ''.join(num_to_char[num] for num in decoded_numbers)
    return decoded_message.replace("?", " ").strip()  # Clean up padding spaces

# Main function
def main():
    print("Matrix Cipher Encoder/Decoder")
    while True:
        choice = input("Choose an operation: (1) Encode, (2) Decode, (3) Exit: ").strip()
        if choice == "1":
            # Encoding
            message = input("Enter the message to encode: ").strip().upper()
            print("Converting the input to uppercase")
            print(message)
            size = int(input("Enter the size of the key matrix (e.g., 2 for 2x2): "))
            print(f"Enter the elements of the {size}x{size} key matrix (one element at a time):")
            key_matrix = [int(input(f"Element [{i+1},{j+1}]: ")) for i in range(size) for j in range(size)]
            encoded_message = encode_message(message, key_matrix)
            print("Encoded message:", encoded_message)
        elif choice == "2":
            # Decoding
            encoded_message = list(map(int, input("Enter the encoded message (space-separated numbers): ").strip().split()))
            size = int(input("Enter the size of the key matrix (e.g., 2 for 2x2): "))
            print(f"Enter the elements of the {size}x{size} key matrix (one element at a time):")
            key_matrix = [int(input(f"Element [{i+1},{j+1}]: ")) for i in range(size) for j in range(size)]
            is_inverse_given = input("Is this the inverse matrix? (y/n): ").strip().lower() == "y"
            try:
                decoded_message = decode_message(encoded_message, key_matrix, is_inverse_given)
                print("Decoded message:", decoded_message)
            except ValueError as e:
                print(f"Error: {e}")
        elif choice == "3":
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Matrix Cipher Encoder/Decoder
Choose an operation: (1) Encode, (2) Decode, (3) Exit: 1
Enter the message to encode: Hi this is AakASh
Converting the input to uppercase
HI THIS IS AAKASH
Enter the size of the key matrix (e.g., 2 for 2x2): 2
Enter the elements of the 2x2 key matrix (one element at a time):
Element [1,1]: 3
Element [1,2]: 2
Element [2,1]: 1
Element [2,2]: 2
Encoded message: 8 23 0 6 8 23 19 12 2 15 20 26 20 20 7 7 15 1
Choose an operation: (1) Encode, (2) Decode, (3) Exit: 3
Exiting...
